# **Data Collection**

## Objectives

- Fetch data from Kaggle, save as raw data and prepare it for further processes.

## Inputs

- Kaggle JSON file - the token is required for kaggle authentication. 

## Outputs

- Generate the Dataset: inputs/cherry-leaves_dataset.

## Additional Comments

- No additional comments here.



---

# Import packages

In [1]:
%pip install -r /workspace/mildew-detector-pp5/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 224.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully uninstalled flatbuffers-24.3.25
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalle

In [2]:
import numpy
import os
import random

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detector-pp5/jupyter_notebooks'

Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detector-pp5/jupyter_notebooks'

# Install Kaggle

Section 1 content

In [5]:
%pip install kaggle==1.5.12

Note: you may need to restart the kernel to use updated packages.


---

In [6]:
import numpy
import os
import random

---


 ## Change working directory

* Notebooks are stored in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder

* We access the current directory with os.getcwd()

In [7]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detector-pp5/jupyter_notebooks'


We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [8]:

os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory



Confirm the new current directory

In [9]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detector-pp5'

---